In [ ]:
import os
import cv2
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from ultralytics import YOLO
from PIL import ImageFont, ImageDraw, Image
from google.colab import drive, files
from IPython.display import display, HTML

# =========================================================
# 1. 환경 설정 및 라이브러리 초기화
# =========================================================
def setup_environment():
    """구글 코랩 환경 및 필요한 라이브러리 설치/확인을 수행합니다."""
    print("🚀 시스템 환경 설정 중...")

    # 드라이브 마운트
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # 한글 폰트 경로 확인 (코랩 기준)
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
    if not os.path.exists(font_path):
        print("⚠️ 한글 폰트가 설치되지 않았습니다. 설치 코드를 먼저 실행해 주세요.")

    return font_path

# =========================================================
# 2. 커플 궁합 해석 엔진
# =========================================================
class CompatibilityInterpreter:
    def __init__(self):
        # 5단계 세분화 텍스트 데이터베이스
        self.texts = {
            'tier_1': [ # 95~100점
                "🦄 <b>[전생에 부부? 영혼의 소울메이트]</b><br>소름 돋는 일치율입니다! 감정의 주파수가 완벽하게 일치하여, 눈빛만 봐도 상대가 무엇을 원하는데 알 수 있는 경지입니다.",
                "👑 <b>[퍼펙트 매치: 운명의 데스티니]</b><br>사랑을 주고받는 방식이 놀랍도록 비슷합니다. 서로의 부족한 점을 채워주는 이상적인 커플입니다."
            ],
            'tier_2': [ # 80~94점
                "🍯 <b>[꿀 떨어지는 사이: 환상의 케미]</b><br>아주 훌륭한 궁합입니다. 서로의 감정선이 부드럽게 조화를 이루어 큰 다툼 없이 예쁜 사랑을 키워갈 수 있습니다.",
                "🌹 <b>[로맨틱 성공적: 든든한 내 편]</b><br>안정감과 설렘을 동시에 느낄 수 있는 사이입니다. 굳건한 신뢰를 바탕으로 롱런할 수 있는 단단한 관계입니다."
            ],
            'tier_3': [ # 60~79점
                "⚖️ <b>[상호보완의 미학: 달라서 더 좋은 사이]</b><br>서로 다른 매력이 톱니바퀴처럼 맞물려 돌아갑니다. 서로에게 없는 점을 배우며 맞춰가는 과정이 사랑을 더 깊게 만듭니다.",
                "🧩 <b>[퍼즐 조각 맞추기: 맞춰가는 즐거움]</b><br>너무 똑같아서 지루하지도, 너무 달라서 힘들지도 않은 딱 좋은 밸런스입니다."
            ],
            'tier_4': [ # 40~59점
                "🔥 <b>[달콤살벌한 로맨스: 자석 같은 끌림]</b><br>정반대의 성향을 가졌기에 강렬하게 끌립니다. '다름'을 인정한다면 영화 같은 사랑을 할 수 있습니다.",
                "🎢 <b>[롤러코스터 사랑: 스릴 넘치는 관계]</b><br>감정선의 흐름이 사뭇 다릅니다. 하지만 지루할 틈이 없다는 것이 두 분 관계의 큰 장점입니다."
            ],
            'tier_5': [ # 0~39점
                "⚡ <b>[치명적 스파크: 화성 남자 금성 여자]</b><br>완전히 다른 행성에서 온 두 분! 서로를 개조하려 하지 말고 있는 그대로 받아들일 때 기적이 일어납니다.",
                "🌈 <b>[신세계 개척: 나를 깨는 사랑]</b><br>상대방은 내가 경험해보지 못한 새로운 세상입니다. 낯섦이 주는 설렘을 즐겨보세요."
            ]
        }

    def calculate_match(self, feat1, feat2):
        """두 손금 특징점을 비교하여 궁합 점수를 계산합니다."""
        # 1. 곡률 차이 (가중치 50%)
        score_curv = max(0, 100 - (abs(feat1['curv'] - feat2['curv']) * 400))
        # 2. 길이 차이 (가중치 30%)
        score_len = max(0, 100 - (abs(feat1['len_ratio'] - feat2['len_ratio']) * 150))
        # 3. 기울기 차이 (가중치 20%)
        score_slope = max(0, 100 - (abs(feat1['slope'] - feat2['slope']) * 50))

        total_score = (score_curv * 0.5) + (score_len * 0.3) + (score_slope * 0.2)

        # 점수 보정 및 최종 산출
        final_score = int(total_score)
        if final_score < 10: final_score += random.randint(10, 20)
        return min(99, final_score)

    def get_result_html(self, score):
        """점수에 따른 결과 리포트 HTML을 생성합니다."""
        if score >= 95: tier, title, color = 'tier_1', "천생연분", "#a55eea"
        elif score >= 80: tier, title, color = 'tier_2', "환상의 짝꿍", "#ff4757"
        elif score >= 60: tier, title, color = 'tier_3', "찰떡궁합", "#2ed573"
        elif score >= 40: tier, title, color = 'tier_4', "달콤살벌", "#ffa502"
        else: tier, title, color = 'tier_5', "치명적 끌림", "#1e90ff"

        desc = random.choice(self.texts[tier])

        return f"""
        <div style="background: #f8f9fa; padding: 30px; border-radius: 30px; text-align: center; border: 4px solid {color}; margin-top: 20px;">
            <div style="font-size: 3em;">💘</div>
            <div style="font-size: 2em; font-weight: bold; color: #2f3542;">{title}</div>
            <div style="font-size: 5em; font-weight: 900; color: {color};">{score}%</div>
            <div style="font-size: 1.1em; color: #57606f; padding: 20px; background: white; border-radius: 15px;">{desc}</div>
        </div>
        """

# =========================================================
# 3. 비전 분석 및 시각화 클래스
# =========================================================
class CoupleAnalysisVisualizer:
    def __init__(self, model_path, font_path):
        self.model = YOLO(model_path)
        self.interpreter = CompatibilityInterpreter()
        self.mp_hands = mp.solutions.hands.Hands(static_image_mode=True, max_num_hands=1)
        self.font_path = font_path
        self.target_line_id = 2  # 감정선 라벨 ID

    def extract_features(self, image_path):
        """이미지에서 손금 특징을 추출합니다."""
        img = cv2.imread(image_path)
        if img is None: return None, None

        # 1. MediaPipe로 손 크기 기준 측정
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mp_res = self.mp_hands.process(img_rgb)
        h, w = img.shape[:2]
        hand_scale = h * 0.5

        if mp_res.multi_hand_landmarks:
            lm = mp_res.multi_hand_landmarks[0].landmark
            p0 = np.array([lm[0].x * w, lm[0].y * h])
            p9 = np.array([lm[9].x * w, lm[9].y * h])
            hand_scale = np.linalg.norm(p9 - p0)

        # 2. YOLO 모델로 감정선 검출
        results = self.model.predict(img, conf=0.05, verbose=False)
        best_kpts = None
        if results and results[0].keypoints is not None:
            for i, cls_id in enumerate(results[0].boxes.cls.cpu().numpy()):
                if int(cls_id) == self.target_line_id:
                    best_kpts = results[0].keypoints.xy.cpu().numpy()[i]
                    break

        if best_kpts is None: return img, None

        # 3. 특징량 계산 (곡률, 길이비, 기울기)
        pts = np.array([(int(x), int(y)) for x, y in best_kpts if x > 0])
        if len(pts) < 2: return img, None

        curve_len = np.sum(np.sqrt(np.sum(np.diff(pts, axis=0)**2, axis=1)))
        dist = np.linalg.norm(pts[-1] - pts[0])
        vec = pts[-1] - pts[0]

        features = {
            'len_ratio': curve_len / hand_scale,
            'curv': curve_len / (dist + 1e-6),
            'slope': vec[1] / (vec[0] + 1e-6)
        }

        # 시각화 드로잉
        cv2.polylines(img, [pts], False, (180, 105, 255), 10, cv2.LINE_AA)
        return img, features

    def run_analysis(self, file_list):
        """전체 분석 프로세스를 실행합니다."""
        results = [self.extract_features(f) for f in file_list]

        if any(res[1] is None for res in results):
            print("❌ 감정선을 찾을 수 없는 사진이 포함되어 있습니다.")
            return

        # 결과 시각화
        plt.figure(figsize=(12, 6))
        for i, (img, _) in enumerate(results):
            plt.subplot(1, 2, i+1)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f"Person {'A' if i==0 else 'B'}")
            plt.axis('off')
        plt.show()

        # 점수 계산 및 출력
        score = self.interpreter.calculate_match(results[0][1], results[1][1])
        display(HTML(self.interpreter.get_result_html(score)))

# =========================================================
# 4. 메인 실행부
# =========================================================
if __name__ == "__main__":
    FONT_PATH = setup_environment()
    MODEL_PATH = '/content/drive/MyDrive/num_02.pt'

    if os.path.exists(MODEL_PATH):
        visualizer = CoupleAnalysisVisualizer(MODEL_PATH, FONT_PATH)

        print("\n📸 [Step 1] 본인의 손 사진을 업로드하세요.")
        up1 = files.upload()

        if up1:
            print("\n📸 [Step 2] 상대방의 손 사진을 업로드하세요.")
            up2 = files.upload()

            if up2:
                file_list = [list(up1.keys())[0], list(up2.keys())[0]]
                visualizer.run_analysis(file_list)
    else:
        print(f"❌ 모델 파일을 찾을 수 없습니다: {MODEL_PATH}")